In [1]:
import pandas as pd
import numpy as np
import scipy

In [2]:
% precision 10

u'%.10f'

In [3]:
pd.set_option('precision',10)

In [5]:
print len(df)
print len(df_filter)

1072997
936680


In [9]:
df.columns

Index([u'NPI', u'NPPES_PROVIDER_LAST_ORG_NAME', u'NPPES_PROVIDER_FIRST_NAME', u'NPPES_PROVIDER_MI', u'NPPES_CREDENTIALS', u'NPPES_PROVIDER_GENDER', u'NPPES_ENTITY_CODE', u'NPPES_PROVIDER_STREET1', u'NPPES_PROVIDER_STREET2', u'NPPES_PROVIDER_CITY', u'NPPES_PROVIDER_ZIP5', u'NPPES_PROVIDER_ZIP4', u'NPPES_PROVIDER_STATE', u'NPPES_PROVIDER_COUNTRY', u'SPECIALTY_DESCRIPTION', u'DESCRIPTION_FLAG', u'MEDICARE_PRVDR_ENROLL_STATUS', u'BENE_COUNT', u'TOTAL_CLAIM_COUNT', u'TOTAL_DRUG_COST', u'TOTAL_DAY_SUPPLY', u'BENE_COUNT_GE65', u'BENE_COUNT_GE65_SUPPRESS_FLAG', u'TOTAL_CLAIM_COUNT_GE65', u'GE65_SUPPRESS_FLAG', u'TOTAL_DRUG_COST_GE65', u'TOTAL_DAY_SUPPLY_GE65', u'BRAND_CLAIM_COUNT', u'BRAND_SUPPRESS_FLAG', u'BRAND_DRUG_COST', u'GENERIC_CLAIM_COUNT', u'GENERIC_SUPPRESS_FLAG', u'GENERIC_DRUG_COST', u'OTHER_CLAIM_COUNT', u'OTHER_SUPPRESS_FLAG', u'OTHER_DRUG_COST', u'MAPD_CLAIM_COUNT', u'MAPD_SUPPRESS_FLAG', u'MAPD_DRUG_COST', u'PDP_CLAIM_COUNT', u'PDP_SUPPRESS_FLAG', u'PDP_DRUG_COST', u'LIS_CLAIM_

In [4]:
df = pd.read_table('PartD_Prescriber_PUF_NPI_14.txt')
df_filter = df[np.isfinite(df['BENE_COUNT'])]   # Counts fewer than 11 are suppressed and are indicated by a blank.
print '%.7f' %  df_filter.BENE_COUNT.mean()

148.4730132


In [11]:
No_sp = df.loc[~df.BRAND_SUPPRESS_FLAG.isin(['*', '#'])]
spe = No_sp.groupby('SPECIALTY_DESCRIPTION').sum()[['BRAND_CLAIM_COUNT','TOTAL_CLAIM_COUNT']].reset_index()
spe_filter = spe.loc[(spe.TOTAL_CLAIM_COUNT >= 1000) ]
spe_filter['frac']  = spe_filter.BRAND_CLAIM_COUNT / spe_filter.TOTAL_CLAIM_COUNT
print '%.9f' %  np.std(spe_filter.frac)

0.087709510


C:\Anaconda\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [105]:
age65 = df.loc[(~df.LIS_SUPPRESS_FLAG.isin(['*', '#'])) & (~df.GE65_SUPPRESS_FLAG.isin(['*', '#'])) ]
age_frac = age65.TOTAL_CLAIM_COUNT_GE65 / age65.TOTAL_CLAIM_COUNT  
low_frac = age65.LIS_CLAIM_COUNT/ age65.TOTAL_CLAIM_COUNT  
from scipy.stats.stats import pearsonr   
print pearsonr(low_frac,age_frac)

(-0.66352487675201421, 0.0)


In [6]:
df['length'] = df.TOTAL_DAY_SUPPLY / df.TOTAL_CLAIM_COUNT
print '%.8f' %  np.median(df.length)

29.10379839


In [13]:
state = df.groupby('NPPES_PROVIDER_STATE').sum()[['OPIOID_BENE_COUNT','ANTIBIOTIC_BENE_COUNT']]
state.opioid_antibiotics = state.OPIOID_BENE_COUNT/  state.ANTIBIOTIC_BENE_COUNT
print '%.9f' % (max(state.opioid_antibiotics) - min(state.opioid_antibiotics) )

0.753386662


In [33]:
spe_state = df.groupby(['SPECIALTY_DESCRIPTION',\
                        'NPPES_PROVIDER_STATE'])[['OPIOID_DAY_SUPPLY','OPIOID_CLAIM_COUNT']].agg(['count','sum']).reset_index()
spe_state.columns = ['SPECIALTY_DESCRIPTION','NPPES_PROVIDER_STATE','OPIOID_DAY_SUPPLY_count',\
                    'OPIOID_DAY_SUPPLY_sum','OPIOID_CLAIM_COUNT_count','OPIOID_CLAIM_COUNT_sum']
spe_state100 = spe_state.loc[spe_state['OPIOID_DAY_SUPPLY_count'] >=100]
special =  spe_state100.groupby('SPECIALTY_DESCRIPTION')[['OPIOID_DAY_SUPPLY_sum','OPIOID_CLAIM_COUNT_sum']].sum().reset_index()
spe_state100['Length_spe_state']  = spe_state100.OPIOID_DAY_SUPPLY_sum / spe_state100.OPIOID_CLAIM_COUNT_sum
special['Length_spe'] = special.OPIOID_DAY_SUPPLY_sum / special.OPIOID_CLAIM_COUNT_sum
result = pd.merge(spe_state100[['SPECIALTY_DESCRIPTION','Length_spe_state']], special[['SPECIALTY_DESCRIPTION','Length_spe']], 
                  on='SPECIALTY_DESCRIPTION')
result['ratio'] = result.Length_spe_state / result.Length_spe
print result.sort('ratio',ascending=False).ratio[:1]

In [160]:
df13 = pd.read_table('PartD_Prescriber_PUF_NPI_13.tab')
df['cost_14'] = df.TOTAL_DRUG_COST / df.TOTAL_DAY_SUPPLY
df13['cost_13'] = df13.TOTAL_DRUG_COST / df13.TOTAL_DAY_SUPPLY
cost = pd.merge(df[['NPI','cost_14']],df13[['NPI','cost_13']],on='NPI',how='inner')
cost['inflation'] =  (cost['cost_14'] - cost['cost_13']) / cost['cost_13']
print cost.inflation.mean()

0.244518352442


In [179]:
merge=pd.merge(df13[['NPI','SPECIALTY_DESC']],df[['NPI','SPECIALTY_DESCRIPTION']],on='NPI',how='left')
merge['left'] = merge.SPECIALTY_DESCRIPTION.isnull().astype(int)
merge_count= merge.groupby('SPECIALTY_DESC').left.agg(['count','sum']).reset_index()
merge_1000 = merge_count.loc[merge_count['count'] >= 1000]
merge_1000['ratio'] = merge_1000['sum'] /merge_count['count'] 
print merge_1000.sort('ratio',ascending=False).ratio[:1]